In [33]:
#! pip install torch torchvision
import torch 
import torchvision
import torchvision.transforms as transforms
from sklearn.metrics import accuracy_score

In [34]:

transform = transforms.Compose([ transforms.ToTensor () ,
                            transforms.Normalize ((0.5 , 0.5 , 0.5) , (0.5 , 0.5 , 0.5) ) ])

In [35]:

trainset = torchvision . datasets .CIFAR10( root='./data', train=True ,
            download=True , transform=transform ) 
testset = torchvision . datasets .CIFAR10( root='./data', train=False ,
            download=True , transform=transform )

Files already downloaded and verified
Files already downloaded and verified


Using the CIFAR 10 dataset 
https://www.cs.toronto.edu/~kriz/cifar.html

In [36]:
import pickle
import os
import numpy as np

<b> File Read And Processing Operations </b>

In [37]:
data_path = "./data/"
img_size = 32
num_channels = 3
img_size_flat = 3072
num_classes = 10
_num_files_train = 5
# Number of images for each batch-file in the training-set.
_images_per_file = 10000
_num_images_train = _num_files_train * _images_per_file

In [38]:
def _get_file_path(filename=""):
    return os.path.join(data_path, "cifar-10-batches-py/", filename)

In [47]:
def _unpickle(filename):
    file_path = _get_file_path(filename)
    print("Loading data: " + file_path)
    with open(file_path, mode='rb') as file:
        data = pickle.load(file)
    return data


In [40]:
def _convert_images(raw):
    raw_float = np.array(raw,dtype=float)/255
    images = raw_float.reshape([-1, num_channels, img_size, img_size])
    images = images.transpose([0,2,3,1])
    return images

In [41]:
def one_hot_encoded(class_numbers, num_classes=None):
    if num_classes is None:
        num_classes = np.max(class_numbers) + 1
    return np.eye(num_classes, dtype=float)[class_numbers]

In [42]:
def _load_data(filename):
    data = _unpickle(filename)
    images = data[b'data']
    cls = np.array(data[b'labels'])
    images = _convert_images(images)
    return images, cls

In [43]:
def load_class_names():
    raw = _unpickle(filename="batches.meta")[b'label_names']
    names = [x.decode('utf-8') for x in raw]
    return names

In [44]:
def load_training_data():
    images = np.zeros(shape=[_num_images_train, img_size, img_size, num_channels], dtype=float)
    cls = np.zeros(shape=[_num_images_train], dtype=int)
    begin = 0
    # For each data-file.
    for i in range(_num_files_train):
        # Load the images and class-numbers from the data-file.
        images_batch, cls_batch = _load_data(filename="data_batch_" + str(i + 1))
        # Number of images in this batch.
        num_images = len(images_batch)
        # End-index for the current batch.
        end = begin + num_images
        print(end)
        # Store the images into the array.
        images[begin:end, :] = images_batch
        # Store the class-numbers into the array.
        cls[begin:end] = cls_batch
        # The begin-index for the next batch is the current end-index.
        begin = end
    return images, cls, one_hot_encoded(class_numbers = cls, num_classes=num_classes)
    

In [45]:
def load_test_data():
    images, cls = _load_data(filename="test_batch")
    return images, cls, one_hot_encoded(class_numbers = cls, num_classes=num_classes)

In [48]:
class_names = load_class_names()
class_names

Loading data: ./data/cifar-10-batches-py/batches.meta


[u'airplane',
 u'automobile',
 u'bird',
 u'cat',
 u'deer',
 u'dog',
 u'frog',
 u'horse',
 u'ship',
 u'truck']

In [49]:
images_train, cls_train, labels_train = load_training_data()

Loading data: ./data/cifar-10-batches-py/data_batch_1
10000
Loading data: ./data/cifar-10-batches-py/data_batch_2
20000
Loading data: ./data/cifar-10-batches-py/data_batch_3
30000
Loading data: ./data/cifar-10-batches-py/data_batch_4
40000
Loading data: ./data/cifar-10-batches-py/data_batch_5
50000


In [50]:
images_test, cls_test, labels_test = load_test_data()

Loading data: ./data/cifar-10-batches-py/test_batch


## We have to reshape the images for the purpose of inputting it to our Neural Network

<h3>Reshaping the images</h3>

In [51]:
images_train = images_train.reshape(50000,-1) # -1 indicates reshaping with the leftover dimensions
cls_train = cls_train.reshape(50000,-1)

images_test = images_test.reshape(10000,-1)
cls_test = cls_test.reshape(10000,-1)

print('Post Reshaping Dimenions..')
print('Training data size- Data,Labels {},{}'.format(images_train.shape,cls_train.shape))
print('Test data size- Data,Labels {},{}'.format(images_test.shape,cls_test.shape))

Post Reshaping Dimenions..
Training data size- Data,Labels (50000, 3072),(50000, 1)
Test data size- Data,Labels (10000, 3072),(10000, 1)


<h3>Neural Network Implementation </h3>

In [53]:
number_of_images = 10000
images_trainset = images_train[:number_of_images].T
labels_trainset = labels_train[:number_of_images]
images_trainset.shape

(3072, 10000)

# Paramters for the neural network

In [54]:
numberOfLayers = 3
neurons_layer_one = 250
neurons_layer_two = 100
neurons_output_layer = 10;

layer_dim = [images_trainset.shape[0],neurons_layer_one,neurons_layer_two,neurons_output_layer]
parameters = {
}


# Neural Network Implementation

In [67]:
class NeuralNetwork:
    def __init__(self,dimensions):
        self.layer_dimensions = dimensions;
        np.random.seed(1);
        self.parameters = parameters;
        L = len(layer_dim)
        
        # initializing the weights of the Neural Network
        for i in range(0,L-1):
            self.parameters["W"+str(i+1)] = (np.random.randn(dimensions[i+1],dimensions[i])*0.01)
            self.parameters["b"+str(i+1)] = np.zeros((dimensions[i+1],1)) 
            
                
    def affineForward(self,A_prev,W,b):
        Z = np.dot(W,A_prev)+b
        cache = (A_prev,W,b)
        return Z,cache
    
    
    def activationForward(self, A,layer_no,activation_function="relu"):
        parameters = self.parameters
        W = parameters["W"+str(layer_no)]
        b = parameters["b"+str(layer_no)]
        Z,linear_cache = self.affineForward(A,W,b)
        if(activation_function == "relu"):
            A,activation_cache = relu(Z)
        elif (activation_function == "softmax"):
            A,activation_cache = sigmoid(Z)
        cache = (linear_cache,activation_cache)
        return A,cache
        

    def forwardPropagation(self,X):
        L = len(self.layer_dimensions)
        A = X
        caches =[]
        for i in range(1,L-1):
            A_prev = A
            A,cache = self.activationForward(A_prev,i,"relu")         
            caches.append(cache)     
        Al,cache = self.activationForward(A,L-1,"softmax")
        caches.append(cache)
        return Al,caches
    
    def costFunction(self,Al,y):
        Al = Al.T
        m = y.shape[0]
        cost = -(1/m)*((np.sum((y*np.log(Al)))+((1-y)*(np.log(1-Al)))))
        return cost
    
                                     #BackPropagation logic begin here
    
    def affineBackward(self,dZ,linear_cache):
        A_prev,W,b = linear_cache
        m = A_prev.shape[1]
        dW = (1/m)*np.dot(dZ,A_prev.T)
        db = (1/m)*np.sum(dZ,axis=1,keepdims=True)
        dA_prev = np.dot(W.T,dZ)
        return dA_prev,dW,db
        
    
    def activationBackward(self, dA, cache,activation_fn):
        linear_cache,activation_cache = cache;
        if activation_fn == "relu":
            dZ = relu_gradient(dA,activation_cache)
            dA_prev, dW,db = self.affineBackward(dZ,linear_cache)
        elif activation_fn == "sigmoid":
            dZ = sigmoid_gradient(dA,activation_cache)
            dA_prev, dW,db = self.affineBackward(dZ,linear_cache)
        return dA_prev, dW,db
    
    
    def backwardPropagation(self,Al,y,caches):
        L = len(caches)
        gradients ={}
        dAl = np.divide(Al - y.T, np.multiply(Al, 1 - Al))
        gradients["dA"+str(L-1)],gradients["dW"+str(L)],gradients["db"+str(L)] = \
                self.activationBackward(dAl,caches[L-1],"sigmoid")
        # Now backpropagation for every layer
        for i in range(L-1,0,-1):
            current_cache = caches[i - 1]
            gradients["dA"+str(i-1)],gradients["dW"+str(i)],gradients["db"+str(i)] = \
                    self.activationBackward(gradients["dA"+str(i)],current_cache,"relu")
        return gradients  
    
    def update_parameters(self, gradients, learning_rate):
        parameters = self.parameters
        L = len(parameters)
        for l in range (1,4):
            parameters["W" + str(l)] = parameters[
                    "W" + str(l)] - learning_rate * gradients["dW" + str(l)]
            parameters["b" + str(l)] = parameters[
                    "b" + str(l)] - learning_rate * gradients["db" + str(l)]
        return parameters
      
    def train(self, X,y, number_of_iterations,learning_rate,activation_fn):
        for i in range (1,number_of_iterations):
            Al,caches = self.forwardPropagation(X);
            Y_pred = Al
            cost = self.costFunction(Al,y)
            if i % 100 == 0:
                print ("Cost: ", cost.mean())
            
            gradients = self.backwardPropagation(Al,y,caches)
            paramters = self.update_parameters(gradients,learning_rate)
            score = accuracy_score(cls_train[:10000],np.argmax(Y_pred.T,axis=1),normalize=True) *100;
        return Y_pred,paramters
    
    def predict(self,X):
        y_pred,caches = self.forwardPropagation(X);
        return y_pred     

In [68]:
# Activation Functions
def relu(Z):
    A = np.maximum(0,Z)
    return A,Z
def leaky_relu(Z):
    A = np.maximum(0.1*Z,Z)
    return A,Z
def sigmoid(Z):
    A = 1/(1+np.exp(-Z))
    return A,Z
def tanh(Z):
    A = (np.exp(Z)-np.exp(-Z))/(np.exp(Z)+np.exp(-Z))
    return A,Z
def softmax(Z):
    Z = Z.T
    A = (np.exp(Z))/float(sum(np.exp(Z)))
    return A,Z

### Derivative Functions

In [69]:
def sigmoid_gradient(dA,Z):
    A,Z = sigmoid(Z)
    dZ = dA * A * (1-A)
    return dZ

def tanh_gradient(dA, Z):
    A, Z = tanh(Z)
    dZ = dA * (1 - np.square(A))

    return dZ
def relu_gradient(dA, Z):
    A, Z = relu(Z)
    dZ = np.multiply(dA, np.int64(A > 0))
    return dZ  

In [70]:
model = NeuralNetwork(layer_dim)
print(model.parameters["W1"].shape,model.parameters["W2"].shape,model.parameters["W3"].shape)
print(model.parameters["b1"].shape,model.parameters["b2"].shape,model.parameters["b3"].shape)

((250, 3072), (100, 250), (10, 100))
((250, 1), (100, 1), (10, 1))


In [71]:
images_trainset.shape

(3072, 10000)

<b> Training the model </b>

In [72]:
Y_pred,paramters = model.train(images_trainset,labels_trainset,7000, 0.03,"relu")

('Cost: ', 0.0)
('Cost: ', 0.0)
('Cost: ', 0.0)
('Cost: ', 0.0)
('Cost: ', 0.0)
('Cost: ', 0.0)
('Cost: ', 0.0)


KeyboardInterrupt: 

In [47]:
from sklearn.metrics import accuracy_score
score = accuracy_score(cls_train[:10000],np.argmax(Y_pred.T,axis=1),normalize=True) *100;
print('Accuracy -----Training Data : {}%'.format(score))

Accuracy -----Training Data : 57.69%


In [48]:
X = images_test.T
X.shape

(3072, 10000)

<b>Predictions</b>

In [51]:
y_pred = model.predict(X)
y_pred = y_pred.T

In [58]:
y_true = cls_test
Y_pred_max = np.argmax(y_pred,axis=1)

In [61]:
np.savetxt('predictions.npy', Y_pred_max)

In [62]:
score = accuracy_score(y_true,Y_pred_max,normalize=True)*100
print('Accuracy for Test Data..... : {}%'.format(score))

Accuracy for Test Data..... : 42.699999999999996%


The network can be made up of different number of layers and by specifying the number of neurons in those layers. Initial results showed that training the network for at least 1000 loops started giving meaningful results. Increasing the neurons increase the accuracy by a lot. After a learning rate was decided upon, the network ran for 7000 loops after which the training accuracy went up to 57% while the test accuracy was around 43%.